In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold

# 將train.csv讀入pandas的DataFrame
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
# 從train將圖像取出，放至DataFrame物件
train_x = train.drop(['label'], axis=1)
# 從train取出正確答案
train_y = train['label']
# 將test.csv讀入pandas的DataFrame
test_x = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

# 將train的資料分為訓練資料與驗證資料
kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

# 將圖像像素值除以255.0，限制在0 ~ 1.0，並轉換為numpy.array
tr_x, va_x = np.array(tr_x / 255.0), np.array(va_x / 255.0)

# 將二維矩陣變換成(高度 = 28, 寬度 = 28, 通道 = 1) 四維矩陣
# 灰階圖片的通道值為1
tr_x = tr_x.reshape(-1,28,28,1)
va_x = va_x.reshape(-1,28,28,1)

# 將正確答案轉換為One-hot encoding
tr_y = to_categorical(tr_y, 10) # numpy.ndarray
va_y = to_categorical(va_y, 10) # numpy.ndarray

# 輸出x_train、y_train的形狀
print(tr_x.shape)
print(tr_y.shape)
print(va_x.shape)
print(va_y.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense, Flatten
# 建立Sequential物件
model = Sequential()
# 卷積層
model.add(Conv2D(filters=32,              # 過濾器數量 
                 kernel_size=(5, 5),      # 使用5 × 5過濾器 
                 padding='same',          # 填補 
                 input_shape=(28, 28, 1), # 輸入資料的形狀 
                 activation='relu'        # 激活函數為 ReLU
                ))

# 展平層
model.add(Flatten())
# 輸出層 
model.add(Dense(10,                  # 輸出層為10個神經元 
                activation='softmax' # 激活函數為Softmax
               ))
# 編譯模型 
model.compile(loss='categorical_crossentropy', # 用交叉熵誤差作為損失函數
              optimizer='rmsprop',             # 優化器為RMSprop 
              metrics=['accuracy']             # 評價指標為準確率
             )            

# 輸出模型結構
model.summary()


In [ ]:
history = model.fit(tr_x,                  # 訓練資料
                    tr_y,                  # 正確答案
                    epochs=20,             # 訓練次數
                    batch_size=100,        # 批次大小
                    verbose=1,             # 輸出訓練的狀況
                    validation_data=(va_x, va_y)) # 驗證資料


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
# 設定繪圖尺寸 
plt.figure(figsize=(15, 6)) # 將圖形縮小、讓圖形之間保有空間
plt.subplots_adjust(wspace=0.2)
# 在1 × 2格線左方 (1,2,1)範圍內繪圖
plt.subplot(1, 2, 1)
# 描繪出訓練資料的損失（錯誤率）
plt.plot(history.history['loss'],
         linestyle = '--',
         label='training') # 描繪繪出驗證資料的損失（錯誤率） 
plt.plot(history.history['val_loss'],
         label='validation') 
plt.ylim(0, 1)           # y 軸範圍 
plt.legend()             # 顯示圖例 
plt.grid()               # 顯示格線 
plt.xlabel('epoch')      # x 軸標籤 
plt.ylabel('loss')       # y 軸標籤

# 在1 × 2格線右方(1,2,2)範圍內繪圖
plt.subplot(1, 2, 2)
# 描繪出訓練資料的準確率
plt.plot( history.history['accuracy'],
         linestyle = '--',
         label='training',)
# 描繪出驗證資料的準確率
plt.plot(history.history['val_accuracy'],
         label='validation')
plt.ylim(0.5, 1)        # y 軸範圍
plt.legend()            # 顯示圖例
plt.grid()              # 顯示格線
plt.xlabel('epoch')     # x 軸標籤
plt.ylabel('acc')       # y 軸標籤
plt.show()
